## Lending Club Credit Risk Modeling
- In this project, I will build three **machine learning** models to predict the three components of expected loss in the context of **credit risk modeling** at the **Lending Club** (a peer-to-peer credit company): Probability of Default (PD), Exposure at Default (EAD) and Loss Given Default (LGD). The expected loss will be the product of these elements: **Expected Loss (EL) = PD * EAD * LGD.** The PD Model will be used to stablish a credit policy, deciding wheter to grant a loan or not for new applicants (application model) based on their credit scores. The EAD and LGD Models will be used along with the PD Model to estimate the Expected Loss (EL) from each loan in order to assess the capital the institution needs to hold to protect itself against defaults.
- The PD modelling encompasses a binary classification problem with target being 1 in case of non-default and 0 in case of default. A Logistic Regression model will be built. 
- The LGD and EAD modelling encompasses a beta regression problem, that is, a regression task in which the dependent variable is beta distributed.
- The **solution pipeline** is based on the **crisp-dm** framework:
    1. Business understanding.
    2. Data understanding.
    3. Data preparation.
    4. Modelling.
    5. Validation.
    6. Deployment.
- Below, I will introduce **information** about the **company**, the **business problem**, the **project objectives and benefits**, and some important **concepts** to have in mind, given that I am dealing with a credit risk problem.

## Lending Club
- **What is the Lending Club?:**
    - LendingClub is a **peer-to-peer lending platform** that facilitates the borrowing and lending of money directly between individuals, without the need for traditional financial institutions such as banks. The platform operates as an online marketplace, connecting borrowers seeking personal loans with investors willing to fund those loans.

## Business Problem: Managing Default Risks and Optimizing Returns
- **What is the business problem?:**
    - LendingClub faces a significant business challenge related to **managing default risks effectively** while **optimizing returns** for its investors. The platform facilitates peer-to-peer lending, connecting borrowers with investors, and relies on **accurate risk assessments to maintain a sustainable and profitable lending ecosystem.** Thus, the CEO wants us to provide insights about which factors are associated with credit risk in Lending Club's operations, and to construct models capable of predicting the probability of default for new applicants and possible losses on its loans in order to establish a credit policy, deciding when to grant a loan or not for an applicant. An important observation is that the CEO wants these models to be easy to understand. Since our company works on the internet, making customers happy and being clear is really important. So, we need to be able to explain why we decide to approve or deny a loan.
- **Which are the project objectives and benefits?**
    1. Identify the factors associated with credit risk in the form of business insights.
    2. Develop an accurate Probability of Default (PD) Model, constructing a scorecard. This will allow Lending Club to decide wheter to grant a loan or not to a new applicant (application model), based on credit scores.
    3. Develop Exposure at Default (EAD) and Loss Given Default (LGD) Models, to estimate the Expected Loss in loans. This will allow Lending Club to hold sufficient capital to protect itself against default in each loan.
    4. Improve risk management and optimize returns by selecting a cut-off point or creating risk categories to approve loans based on our models' results. With this, we intend to establish a credit policy, trying to balance risk and ROI of Lending Club's assets.
    5. Apply model monitoring and maintenance techniques to safeguard our results from population instability, characterized by significant changes in loan applicants' characteristics. This will allow us to understand whether the built model is still useful in the future or whether the loan applicants characteristics changed significantly, such that we will need to redevelop it.

## Credit Risk Modeling Concepts: The Context of the Problem
- **Financial institutions**, like LendingClub and online lending platforms, **make money by lending to people and businesses.** When they lend money, they **charge interest**, which is a significant source of their **profits**. **Managing credit risk well is crucial** for these institutions. This means ensuring that borrowers pay back their loans on time to avoid losses.
- **Credit risk** is the possibility that a borrower might not fulfill their financial obligations, leading to a loss for the lender. If a borrower fails to meet the agreed-upon terms, it's called a "default," and it can result in financial losses for the lender. The **default** definition is associated with a time horizon. For example, if a borrower hasn't paid their debt within 90 days of the due date, they are considered in default.
- In the credit market, important **rules** help keep things honest and clear. **Basel III** is one such set of rules, making sure banks have **enough money (capital requirements)** and follow **guidelines for assessing loan risks**. The **Internal Rating-Based Approach (IRB-A)** lets banks figure out credit risks using concepts like Probability of Default (PD), Exposure at Default (EAD), and Loss Given Default (LGD). Another rule, **International Financial Reporting Standard 9 (IFRS 9)**, gives standards for measuring financial assets. It's special because it looks at the chance of a loan not being paid back over its entire life, unlike Basel, which checks it for one year. These rules help banks have enough money, handle risks well, and keep the credit market steady and trustworthy.
- The **"expected loss (EL)"** is the average estimated loss that a lender can expect from loans that default. It involves three factors: the **probability of default (likelihood of a borrower defaulting)**, **loss given default (portion of the amount the bank is exposed to that can't be recovered in case of default)**, and **exposure at default (potential loss at the time of default, considering the outstanding loan amount and other factors)**.
- **LendingClub**, operating as a peer-to-peer lending platform, uses a **"PD Model/Credit Scoring Model" to assess borrowers' creditworthiness using credit scores** from a scorecard. This helps determine the **likelihood of loan repayment**, guiding the decision to **approve or deny the loan.** The **required capital to guard against default** for each loan is calculated using **EAD and LGD Models** to estimate the **Expected Loss (EL)**, contributing to minimizing risk in credit operations.
- When creating a Credit Scoring Model, which assesses creditworthiness for loan approval, using data available at the time of the application is considered an **"application model."** It is distinct from a "behavior model." This is the model I will build here.
- A **"credit policy"** is a set of guidelines that financial institutions follow to evaluate and manage lending risk. It involves setting a cut-off point, a threshold based on credit scores, or risk categories, to decide whether to approve or reject loan applications. Another factors such as the expected ROI for each loan application are included.
- **"Return on Investment (ROI)"** is a key measure of loan profitability. Balancing ROI with risk is vital for effective credit policy management. While higher-risk loans may offer more significant potential returns, they also come with a higher chance of default.

In **this notebook** I intend to perform an **initial data exploration and data cleaning**, necessary to explore the data and build robust models further, encompassing the first and second steps of crisp-dm framework. The objective here is to convert features to the correct data types, initially remove unuseful variables, treat/diagnose missing, duplicate and outlier values, create new relevant variables and obtain a final memory optimized clean dataset for faster and more efficient analysis. Finally, the rationale behind each decision is explained in detail below.

### 1.0 Importing the libraries

In [1]:
# Data manipulation and visualization.
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

# Display options.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_info_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Filter warnings.
import warnings
warnings.filterwarnings('ignore')

### 1.1 Data Understanding
- The data was collected from kaggle and contain complete loan data for all loans issued through the 2007-2015, including the current loan status (Current, Late, Fully Paid, etc.) and the latest payment information.
- This include consumer loans, as explained above in Lending Club's services, and encompasses data available at the moment of the application. Thus, it will be used to build an application model.
- In this step, I intend to perform some initial data understanding, like assessing the dimensions of the dataset, features data types, missing and duplicated values, categorical and numerical variables and so on.

In [2]:
path = '/Users/pedrohenriquealmeidaoliveira/Documents/data_science/large_files/loan_data_2007_2014.csv'
df = pd.read_csv(path)

In [3]:
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,NaN,Jan-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-96,0.0,38.0,NaN,15.0,0.0,27783,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-16,67.79,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.tail()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
466280,466280,8598660,1440975,18400,18400,18400.0,60 months,14.47,432.64,C,C2,Financial Advisor,4 years,MORTGAGE,110000.0,Source Verified,Jan-14,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,773xx,TX,19.85,0.0,Apr-03,2.0,NaN,NaN,18.0,0.0,23208,77.6,36.0,w,12574.00,12574.00,10383.360000,10383.36,5826.00,4557.36,0.0,0.0,0.0,Jan-16,432.64,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,294998.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29900.0,NaN,NaN,NaN
466281,466281,9684700,11536848,22000,22000,22000.0,60 months,19.97,582.50,D,D5,Chief of Interpretation (Park Ranger),10+ years,MORTGAGE,78000.0,Verified,Jan-14,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,377xx,TN,18.45,0.0,Jun-97,5.0,NaN,116.0,18.0,1.0,18238,46.3,30.0,f,0.00,0.00,4677.920000,4677.92,1837.04,2840.88,0.0,0.0,0.0,Dec-14,17.50,NaN,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,221830.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39400.0,NaN,NaN,NaN
466282,466282,9584776,11436914,20700,20700,20700.0,60 months,16.99,514.34,D,D1,patrol,7 years,MORTGAGE,46000.0,Verified,Jan-14,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/06/13 > I am going to c...,debt_consolidation,Debt consolidation,458xx,OH,25.65,0.0,Dec-01,2.0,65.0,NaN,18.0,0.0,6688,51.1,43.0,f,14428.31,14428.31,12343.980000,12343.98,6271.69,6072.29,0.0,0.0,0.0,Jan-16,514.34,Feb-16,Dec-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,73598.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13100.0,NaN,NaN,NaN
466283,466283,9604874,11457002,2000,2000,2000.0,36 months,7.90,62.59,A,A4,Server Engineer Lead,3 years,OWN,83000.0,Verified,Jan-14,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,913xx,CA,5.39,3.0,Feb-03,1.0,13.0,NaN,21.0,0.0,11404,21.5,27.0,w,0.00,0.00,2126.579838,2126.58,2000.00,126.58,0.0,0.0,0.0,Dec-14,1500.68,NaN,Apr-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,591610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53100.0,NaN,NaN,NaN
466284,466284,9199665,11061576,10000,10000,9975.0,36 months,19.20,367.58,D,D3,NaN,10+ years,MORTGAGE,46000.0,Verified,Jan-14,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/04/13 > I will like a l...,other,Other,950xx,CA,22.78,1.0,Feb-00,0.0,9.0,NaN,6.0,0.0,11325,70.8,22.0,f,3984.38,3974.41,8821.620000,8799.57,6015.62,2806.00,0.0,0.0,0.0,Jan-16,367.58,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,57477.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16000.0,NaN,NaN,NaN


#### 1.2 Data dictionary
- Below we have the information about what each feature of the dataset means.

**addr_state**: The state provided by the borrower in the loan application

**annual_inc**: The self-reported annual income provided by the borrower during registration.

**annual_inc_joint**: The combined self-reported annual income provided by the co-borrowers during registration.

**application_type**: Indicates whether the loan is an individual application or a joint application with two co-borrowers

**collection_recovery_fee**: Post charge-off collection fee

**collections_12_mths_ex_med**: Number of collections in 12 months excluding medical collections

**delinq_2yrs**: The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years

**desc**: Loan description provided by the borrower

**dti**: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.

**dti_joint**: A ratio calculated using the co-borrowers' total monthly payments on the total debt obligations, excluding mortgages and the requested LC loan, divided by the co-borrowers' combined self-reported monthly income

**earliest_cr_line**: The month the borrower's earliest reported credit line was opened

**emp_length**: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.

**emp_title**: The job title supplied by the Borrower when applying for the loan.

**fico_range_high**: The upper boundary range the borrower’s FICO at loan origination belongs to.

**fico_range_low**: The lower boundary range the borrower’s FICO at loan origination belongs to.

**funded_amnt**: The total amount committed to that loan at that point in time.

**funded_amnt_inv**: The total amount committed by investors for that loan at that point in time.

**grade**: LC assigned loan grade

**home_ownership**: The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.

**id**: A unique LC assigned ID for the loan listing.

**initial_list_status**: The initial listing status of the loan. Possible values are – W, F

**inq_last_6mths**: The number of inquiries in the past 6 months (excluding auto and mortgage inquiries)

**installment**: The monthly payment owed by the borrower if the loan originates.

**int_rate**: Interest Rate on the loan

**is_inc_v**: Indicates if income was verified by LC, not verified, or if the income source was verified

**issue_d**: The month which the loan was funded

**last_credit_pull_d**: The most recent month LC pulled credit for this loan

**last_fico_range_high**: The upper boundary range the borrower’s last FICO pulled belongs to.

**last_fico_range_low**: The lower boundary range the borrower’s last FICO pulled belongs to.

**last_pymnt_amnt**: Last total payment amount received

**last_pymnt_d**: Last month payment was received

**loan_amnt**: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

**loan_status**: Current status of the loan

**member_id**: A unique LC assigned Id for the borrower member.

**mths_since_last_delinq**: The number of months since the borrower's last delinquency.

**mths_since_last_major_derog**: Months since the most recent 90-day or worse rating

**mths_since_last_record**: The number of months since the last public record.

**next_pymnt_d**: Next scheduled payment date

**open_acc**: The number of open credit lines in the borrower's credit file.

**out_prncp**: Remaining outstanding principal for the total amount funded

**out_prncp_inv**: Remaining outstanding principal for the portion of total amount funded by investors

**policy_code**: Publicly available policy_code=1; new products not publicly available policy_code=2

**pub_rec**: Number of derogatory public records

**purpose**: A category provided by the borrower for the loan request.

**pymnt_plan**: Indicates if a payment plan has been put in place for the loan

**recoveries**: Post charge-off gross recovery

**revol_bal**: Total credit revolving balance

**revol_util**: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

**sub_grade**: LC assigned loan subgrade

**term**: The number of payments on the loan. Values are in months and can be either 36 or 60.

**title**: The loan title provided by the borrower

**total_acc**: The total number of credit lines currently in the borrower's credit file

**total_pymnt**: Payments received to date for the total amount funded

**total_pymnt_inv**: Payments received to date for the portion of the total amount funded by investors

**total_rec_int**: Interest received to date

**total_rec_late_fee**: Late fees received to date

**total_rec_prncp**: Principal received to date

**url**: URL for the LC page with listing data

**verified_status_joint**: Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified

**zip_code**: The first 3 numbers of the zip code provided by the borrower in the loan application.

**open_acc_6m**: Number of open trades in the last 6 months

**open_il_6m**: Number of currently active installment trades

**open_il_12m**: Number of installment accounts opened in the past 12 months

**open_il_24m**: Number of installment accounts opened in the past 24 months

**mths_since_rcnt_il**: Months since most recent installment accounts opened

**total_bal_il**: Total current balance of all installment accounts

**il_util**: Ratio of total current balance to high credit/credit limit on all installment accounts

**open_rv_12m**: Number of revolving trades opened in the past 12 months

**open_rv_24m**: Number of revolving trades opened in the past 24 months

**max_bal_bc**: Maximum current balance owed on all revolving accounts

**all_util**: Balance to credit limit on all trades

**total_rev_hi_lim**: Total revolving high credit/credit limit

**inq_fi**: Number of personal finance inquiries

**total_cu_tl**: Number of finance trades

**inq_last_12m**: Number of credit inquiries in the past 12 months

**acc_now_delinq**: The number of accounts on which the borrower is now delinquent.

**tot_coll_amt**: Total collection amounts ever owed

**tot_cur_bal**: Total current balance of all accounts

*Employer Title replaces Employer Name for all loans listed after 9/23/2013*

- At a glance, there are some **variables** that can drive **valuable insights** about **credit risk** in Lending Club's operations:
    - Most of the demographical variables: addr_state, annual_inc, emp_length, home_ownership.
    - From the financial variables: dti, revol_bal, revol_util, tot_cur_bal, total_rev_hi_lim, total_acc.
    - From the credit and risk variables: delinq_2yrs, recoveries, out_prncp, total_rec_prncp, grade, inq_last_6_months and 12m, funded_amnt, int_rate and term.
    - From time-related and other variables: issue_d, purpose.

#### 1.3 General information
- I will assess general information about the data, such as dimensions, features data types, null and duplicated values, and so on in order to initially see some possible problems that will require data cleaning and gather more information about the data I am dealing with.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 75 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Unnamed: 0                   int64  
 1   id                           int64  
 2   member_id                    int64  
 3   loan_amnt                    int64  
 4   funded_amnt                  int64  
 5   funded_amnt_inv              float64
 6   term                         object 
 7   int_rate                     float64
 8   installment                  float64
 9   grade                        object 
 10  sub_grade                    object 
 11  emp_title                    object 
 12  emp_length                   object 
 13  home_ownership               object 
 14  annual_inc                   float64
 15  verification_status          object 
 16  issue_d                      object 
 17  loan_status                  object 
 18  pymnt_plan                   object 
 19  ur

In [6]:
print(f'The dataset has {df.shape[0]} rows and {df.shape[1]} columns.')

The dataset has 466285 rows and 75 columns.


- At a glance, **some variables can be removed.** This include variables with an unique value per observation, like the ids, for example.
- **Some variables are in the incorrect data type** and must be converted to the correct one. It is the case of term and emp_length, which must be converted from object to int. Moreover, issue_d and earliest_cr_line must be converted from object to datetime format, because they are date-related features.

#### 1.4 Missing and Duplicate values
- I will assess the missing values distributions across the variables in order to investigate their motivation and treat them here or not. The same for duplicate values.

In [7]:
missing_df = df.isna().sum().to_frame().rename(columns={0: 'count'}).sort_values(by=['count'], ascending=False)
missing_df['pct'] = round(missing_df['count'] / df.shape[0] * 100, 3)
missing_df

,count,pct
inq_last_12m,466285,100.000
total_bal_il,466285,100.000
dti_joint,466285,100.000
verification_status_joint,466285,100.000
annual_inc_joint,466285,100.000
open_acc_6m,466285,100.000
open_il_6m,466285,100.000
open_il_12m,466285,100.000
open_il_24m,466285,100.000
mths_since_rcnt_il,466285,100.000


In [8]:
df.duplicated().sum()

0

- There are not duplicated values in the data.
- It is possible to see that there are a lot of variables with a 100% of missings. These are not useful for anything and will be removed.
- I will stablish a 70% cut-off to remove variables with missing values. That is, if the variable has 70% or higher missing rate, it will be removed.
- For the other features, I will investigate the missing values occurrence and decide whether to keep them or not. Some considerations:
    - If the **pattern of missing values is not random**, meaning that the occurrence of missing values varies between good and bad loans, for example, if there are proportionally more missing values in bad loans, it is advisable to treat it as a category within the feature. The absence of information from the applicant could be related to their intention to repay or not repay their debt.
    - In case the **pattern of missing values is random, we can:**
        - Remove these observations, in case we have a big sample of applicant data (and we have!) and their frequency is really low.
        - Exclude variables with an excessive number of missing values (which we already considered doing).
        - Keep the missing as one of the categories of the variable, like "unknown". It will have a weight in the final scoring formula.
        - Impute values. Utilizing mean, median, mode or even inferring it from another variable highly linearly correlated with the variable with missings are possible alternatives.
- The occurrence of **missing values** might be due to different **reasons**, like:
    - The applicant didn't gave this information.
    - The given information was not registered.
    - The given information does not match with any of the expected codes or values for that variable, then, it was registered as a missing. This is pretty common in occupation, where some abbreviations come.
    - The missing is, in reality, zero.
- Let's investigate the missing values in relevant features below.

In [9]:
# Investigating missing values in annual income.
df.loc[df['annual_inc'].isna()].sample(3)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
42533,42533,71623,70735,6500,6500,0.0,36 months,8.38,204.84,A,A5,NaN,< 1 year,NONE,NaN,Not Verified,Jun-07,Does not meet the credit policy. Status:Fully ...,n,https://www.lendingclub.com/browse/loanDetail....,"Hi, I'm buying a used car. Anybody on faceb...",other,Buying a car,100xx,NY,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,f,0.0,0.0,7373.83,0.00,6500.0,873.83,0.0,0.0,0.0,Jun-10,205.32,Jul-10,Aug-07,NaN,NaN,1,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42480,42480,79893,79878,6700,6700,6700.0,36 months,7.75,209.18,A,A3,NaN,< 1 year,NONE,NaN,Not Verified,Jul-07,Does not meet the credit policy. Status:Fully ...,n,https://www.lendingclub.com/browse/loanDetail....,I am moving to Florida and would like to borro...,other,Moving expenses and security deposit,100xx,NY,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,f,0.0,0.0,7530.42,7530.42,6700.0,830.42,0.0,0.0,0.0,Jul-10,209.89,Aug-10,May-07,NaN,NaN,1,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42449,42449,79967,79912,5000,5000,3025.0,36 months,7.43,155.38,A,A2,NaN,< 1 year,NONE,NaN,Not Verified,Aug-07,Does not meet the credit policy. Status:Fully ...,n,https://www.lendingclub.com/browse/loanDetail....,I will be relocating to Boston next month and ...,other,Moving expenses,100xx,NY,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,f,0.0,0.0,5593.46,3384.01,5000.0,593.46,0.0,0.0,0.0,Aug-10,157.31,Sep-10,May-07,NaN,NaN,1,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Investigating missing values in total_acc.
df.loc[df['total_acc'].isna()].sample(3)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
42520,42520,83979,83974,3000,3000,250.0,36 months,7.43,93.23,A,A2,NC,1 year,MORTGAGE,80000.0,Not Verified,Jun-07,Does not meet the credit policy. Status:Fully ...,n,https://www.lendingclub.com/browse/loanDetail....,Need to relocate this money will help before I...,other,Moving Expenses for relocation,070xx,NJ,0.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,f,0.0,0.0,3120.26,260.02,3000.00,120.26,0.00,0.00,0.00,Jan-08,0.00,Jan-08,Jun-07,NaN,NaN,1,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42521,42521,83489,83471,2600,2600,575.0,36 months,8.38,81.94,A,A5,College Pro Painters,3 years,MORTGAGE,6500.0,Not Verified,Jun-07,Does not meet the credit policy. Status:Charge...,n,https://www.lendingclub.com/browse/loanDetail....,I would like to refinance my credit card debt ...,other,Credit card refinancing,806xx,CO,6.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,f,0.0,0.0,2342.01,517.27,1962.69,329.05,21.64,28.63,0.34,Mar-10,21.75,Mar-10,Mar-10,NaN,NaN,1,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42527,42527,77757,70626,3000,3000,0.0,36 months,9.33,95.86,B,B3,NaN,< 1 year,OWN,20000.0,Not Verified,Jun-07,Does not meet the credit policy. Status:Fully ...,n,https://www.lendingclub.com/browse/loanDetail....,For home improvement,other,Home improvement,024xx,MA,10.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,f,0.0,0.0,3450.79,0.00,3000.00,450.79,0.00,0.00,0.00,Jun-10,96.77,Jul-10,May-07,NaN,NaN,1,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Assessing missing values loan_status distribution in emp_length to see whether their occurrence is at random.
df.loc[df['emp_length'].isna(), 'loan_status'].value_counts(normalize=True)

loan_status
Current                                                0.556026
Fully Paid                                             0.302313
Charged Off                                            0.110529
Late (31-120 days)                                     0.019088
In Grace Period                                        0.004855
Late (16-30 days)                                      0.003284
Default                                                0.002285
Does not meet the credit policy. Status:Fully Paid     0.000904
Does not meet the credit policy. Status:Charged Off    0.000714
Name: proportion, dtype: float64

In [12]:
# Investigating missing values in emp_length.
df.loc[df['emp_length'].isna()].sample(5)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
112675,112675,6299644,7830979,21000,21000,21000.0,36 months,6.62,644.78,A,A2,NaN,NaN,MORTGAGE,160000.0,Source Verified,Jul-13,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 07/15/13 > Loan is to pay ...,debt_consolidation,lending1,926xx,CA,8.97,1.0,Dec-98,0.0,4.0,NaN,13.0,0.0,137014,17.1,30.0,f,0.00,0.00,22848.629230,22848.63,21000.00,1848.63,0.0,0.0,0.0,Jun-15,9321.15,NaN,May-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,1041934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,299700.0,NaN,NaN,NaN
332003,332003,23503192,25856105,5700,5700,5700.0,36 months,8.39,179.65,A,A5,NaN,NaN,RENT,19344.0,Verified,Aug-14,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,891xx,NV,27.23,0.0,Nov-88,1.0,68.0,112.0,7.0,1.0,5560,28.0,11.0,w,3185.75,3185.75,3054.050000,3054.05,2514.25,539.80,0.0,0.0,0.0,Jan-16,179.65,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,38174.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151.0,NaN,NaN,NaN
217895,217895,1275476,1518923,3600,3600,3600.0,36 months,9.76,115.76,B,B1,NaN,NaN,RENT,25300.0,Source Verified,May-12,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit Card Refinance,481xx,MI,11.34,0.0,Jun-01,0.0,58.0,NaN,14.0,0.0,4467,20.7,35.0,f,0.00,0.00,4092.903576,4092.90,3600.00,492.90,0.0,0.0,0.0,Jan-15,6.94,NaN,Jan-15,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252434,252434,34813623,37386933,20000,20000,20000.0,60 months,10.49,429.78,B,B3,NaN,NaN,OWN,54000.0,Verified,Nov-14,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,248xx,WV,13.40,0.0,Apr-93,0.0,NaN,NaN,4.0,0.0,7808,38.8,12.0,w,16220.67,16220.67,5993.610000,5993.61,3779.33,2214.28,0.0,0.0,0.0,Jan-16,429.78,Feb-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,15220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20100.0,NaN,NaN,NaN
398078,398078,15449851,17522298,10000,10000,10000.0,36 months,7.90,312.91,A,A4,NaN,NaN,MORTGAGE,41000.0,Verified,May-14,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,430xx,OH,17.04,2.0,Jan-87,0.0,14.0,NaN,15.0,0.0,15652,33.9,31.0,w,0.00,0.00,10370.385950,10370.39,10000.00,370.39,0.0,0.0,0.0,Nov-14,8805.84,NaN,Nov-14,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,505.0,149628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46200.0,NaN,NaN,NaN


In [13]:
# Investigating missing values in total current balance, collection amount, and credit limit.
df.loc[df[['tot_cur_bal', 'tot_coll_amt', 'total_rev_hi_lim']].isna().any(axis=1)].sample(3)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
215137,215137,1321169,1566260,4000,4000,4000.0,36 months,18.25,145.12,D,D3,Flextronics,3 years,RENT,28800.0,Source Verified,Jun-12,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,951xx,CA,10.38,0.0,Apr-01,0.0,41.0,NaN,10.0,0.0,8240,64.3,12.0,f,0.0,0.0,3705.590000,3705.59,2412.85,1062.83,0.0,229.91,2.2991,Jun-14,290.24,NaN,Nov-14,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15207,15207,747071,945910,2400,2400,2400.0,36 months,11.99,79.71,B,B5,First Vehicle Services,5 years,RENT,34000.0,Verified,May-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,moving,Relocation expenses,085xx,NJ,9.99,0.0,Nov-98,0.0,NaN,NaN,5.0,0.0,1010,31.6,7.0,f,0.0,0.0,2869.078545,2869.08,2400.00,469.08,0.0,0.00,0.0000,Jun-14,80.22,NaN,May-14,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17163,17163,720966,915467,5200,5200,5200.0,36 months,13.43,176.29,C,C3,NuVision FCU,3 years,RENT,52000.0,Not Verified,Apr-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 04/05/11 > Paying off one ...,debt_consolidation,debt console,926xx,CA,9.51,0.0,May-97,2.0,50.0,NaN,7.0,0.0,946,14.8,18.0,f,0.0,0.0,6342.835817,6342.84,5200.00,1142.84,0.0,0.00,0.0000,Apr-14,191.05,NaN,Aug-14,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- A lot of variables containing missings represent features that will not be available at the time of prediction and thus will not be investigated/treated because they will not be included in the modeling task to prevent data leakage.
- annual_inc, total_acc, acc_now_delinq, pub_rec, open_acc, delinq_2yrs, earliest_cr_line and inq_last_6mths don't seem to present any pattern regard to default and non-default distributions in its missing values. However, it is observed that the loan_status for all these null observations does not meet the credit policy. 
- While it's generally not recommended, in this case, null values in these variables make up a tiny part of the data (29 records, less than 0.0001%). Since I have a large sample size, I've decided to simply remove these observations. This simplifies our data processing, reduces the need for imputation or other treatments for missing values, and won't affect the results due to the substantial size of the dataset.
- There is no discernible pattern in the default/non-default distributions related to missing values in the 'emp_length' variable. Moreover, we can reasonably assume that these instances correspond to consumers without professional experience. As a result, we will substitute these missing values with 0.
- For tot_cur_bal, tot_coll_amt and total_rev_hi_lim, it might be interesting to impute missing values with mean/median, or even the values of a highly linearly correlated independent variable. This will be done further, in the modeling task. Imputing missings before the data split might introduce data leakage.
- All the treatments will be made in the data cleaning topic.

#### 1.5 Numerical and Categorical features
- I'll check which numerical and categorical variables are in the data. This helps me separate them and check how many different categories there are in the categorical features. The number of categories is important because it affects how we encode the data. If there are too many categories, using one-hot encoding can make the data dimensionality too big. This can be a problem. Also, I'll see if a variable is helpful or not. Sometimes, variables have too many categories and don't really help the model. In that case, we might need to remove them from the analysis.

In [14]:
# Assessing which are the numerical and categorical features.
numerical_features = df.select_dtypes('number').columns.tolist()
categorical_features = df.select_dtypes('object').columns.tolist()
print(f'There are {len(numerical_features)} in the data. They are: {numerical_features}.')
print(f'There are {len(categorical_features)} in the data. They are: {categorical_features}.')

There are 53 in the data. They are: ['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m'].
There are 22 in the data. They are: ['term', 'grade', 'sub_grade', 'emp_title', 'emp_length'

In [15]:
# Assessing categorical features cardinality.
'''
for feature in categorical_features:
    print(f'{feature} - cardinality = {df[feature].nunique()}')
    print()
    print(df[feature].unique())
    print('-' * 100)
'''

"\nfor feature in categorical_features:\n    print(f'{feature} - cardinality = {df[feature].nunique()}')\n    print()\n    print(df[feature].unique())\n    print('-' * 100)\n"

In [16]:
# Assessing pymnt_plan distribution in order to see whether there is imbalancement.
df['pymnt_plan'].value_counts(normalize=True)

pymnt_plan
n    0.999981
y    0.000019
Name: proportion, dtype: float64

In [17]:
# Assessing initial_list_status distribution in order to see whether there is imbalancement.
df['initial_list_status'].value_counts(normalize=True)

initial_list_status
f    0.649828
w    0.350172
Name: proportion, dtype: float64

In [18]:
# Assessing policy_code distribution in order to see whether there is imbalancement.
df['policy_code'].value_counts(normalize=True)

policy_code
1    1.0
Name: proportion, dtype: float64

#### 1.6 Data Cleaning and Feature Engineering
- I will apply an initial **data cleaning**, including the following tasks:
    - **Treat missing values:**
        - Null values in annual_inc, total_acc, acc_now_delinq, pub_rec, open_acc, delinq_2yrs, earliest_cr_line and inq_last_6mths make up a tiny part of the data (29 records, less than 0.0001%). Since I have a large sample size, I've decided to simply remove these observations. This simplifies our data processing, reduces the need for imputation or other treatments for missing values, and won't affect the results due to the substantial size of the dataset.
        - There is no discernible pattern in the default/non-default distributions related to missing values in the emp_length variable. Moreover, we can reasonably assume that these instances correspond to consumers without professional experience. As a result, we will substitute these missing values with 0.
    - **Remove irrelevant variables:**
        - ids, url, desc, next_pymnt_d, last_pymnt_d and last_credit_pull_d will be removed because they are unuseful for the modeling or the analysis.
        - application_type, pymnt_plan and policy_code will be removed because they are almost constant or constant, presenting no variance.
        - zip_code, emp_title and title will be removed becasuse they present excessive cardinality.
        - Variables with a missing rate higher than 70% will be removed in order to avoid bias.
        - As mentioned above, variables like loan_amnt will not be available at the moment of prediction (production env) and thus will be removed to prevent from data leakage. However, once they can present valuable insights about credit risk in Lending Club's operations, I will do this in the data preprocessing step, after exploring them.
    - **Convert variables to the correct data types:**
        - Convert term and emp_length from object to int data type.
        - Convert date related features from object to datetime data type. Considering that some features contain dates around 2016 and 2017, I will use 2017-12-01 as the current date for this feature extraction.
- From the features I have, I will try to **extract new features**, obtaining the maximum amount of information from the available variables.
    - default: This will be our target variable, indicating whether the consumer defaulted (0) or not (1). I will choose different loan_status categories to insert in default/non-default.
        - I won't consider the grace period or late payments between 16 and 30 days in my definition of defaults. This is because, for customers falling into these categories, there's still a good chance they will repay their loans. It's too soon to label them as defaulters.
        - I chose to represent non-defaulters as 1 and defaulters as 0. This is because, when interpreting our model, I want the coefficients to reflect a negative impact for defaulters, resulting in lower values.
    - mths_since_earliest_cr_line: Self explanatory. It is interesting for us to obtain a numerical variable indicating the number of months since the earliest credit line.

In [19]:
# Creating a copy to saveguard the original data.
clean_df = df.copy()

In [20]:
# Removing irrelevant variables.
to_drop_missing = missing_df.loc[missing_df['pct'] >= 70].index.tolist()
to_drop_irrelevant = ['id', 'member_id', 'url', 
                      'Unnamed: 0', 'next_pymnt_d', 
                      'application_type', 'pymnt_plan', 
                      'policy_code', 'zip_code', 'title',
                      'emp_title', 'last_pymnt_d', 'last_credit_pull_d']
to_drop = to_drop_missing + to_drop_irrelevant
clean_df = clean_df.drop(columns=to_drop)
clean_df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim'], dtype='object')

In [21]:
# Treating missing values.
to_drop_missings = ['annual_inc', 'total_acc', 'acc_now_delinq', 'pub_rec', 'open_acc', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths']
clean_df = clean_df.dropna(subset=to_drop_missings)
clean_df['emp_length'] = clean_df['emp_length'].fillna('0 years')
clean_df.shape

(466256, 42)

In [22]:
# Converting variables to the correct data type.
clean_df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [23]:
# Converting term variable to int format.
clean_df['term'] = clean_df['term'].apply(lambda x: int(x[1:3]))
clean_df['term'].unique(), clean_df.term.dtype

(array([36, 60]), dtype('int64'))

In [24]:
# Assessing emp_length unique values to correct the data type.
clean_df['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', '0 years'],
      dtype=object)

In [25]:
# Converting emp_length to float format because of the missing values.
clean_df['emp_length'] = clean_df['emp_length'].str.replace('+', '')
clean_df['emp_length'] = clean_df['emp_length'].str.replace('< 1 year', '0')
clean_df['emp_length'] = clean_df['emp_length'].str.replace(' years', '')
clean_df['emp_length'] = clean_df['emp_length'].str.replace(' year', '')
clean_df['emp_length'] = clean_df['emp_length'].astype('int')
clean_df['emp_length'].unique()

array([10,  0,  1,  3,  8,  9,  4,  5,  6,  2,  7])

In [26]:
# Assessing some earliest_cr_line values to correct the data type.
clean_df['earliest_cr_line'].head()

0    Jan-85
1    Apr-99
2    Nov-01
3    Feb-96
4    Jan-96
Name: earliest_cr_line, dtype: object

In [27]:
# Converting earlist_cr_line to datetime format.
clean_df['earliest_cr_line'] = pd.to_datetime(clean_df['earliest_cr_line'], format='%b-%y')
clean_df['earliest_cr_line'].head()

0   1985-01-01
1   1999-04-01
2   2001-11-01
3   1996-02-01
4   1996-01-01
Name: earliest_cr_line, dtype: datetime64[ns]

In [28]:
# Assessing some issue_d values to correct the data type.
clean_df['issue_d'].head()

0    Dec-11
1    Dec-11
2    Dec-11
3    Dec-11
4    Dec-11
Name: issue_d, dtype: object

In [29]:
# Converting issue_d to datetime format.
clean_df['issue_d'] = pd.to_datetime(clean_df['issue_d'], format='%b-%y')
clean_df['issue_d'].head()

0   2011-12-01
1   2011-12-01
2   2011-12-01
3   2011-12-01
4   2011-12-01
Name: issue_d, dtype: datetime64[ns]

In [30]:
# Extracting new features
CURR_DATE = pd.to_datetime('2017-12-01')

# Extracting mths_since_earliest_cr_line variable.
clean_df['mths_since_earliest_cr_line'] = round((CURR_DATE - clean_df['earliest_cr_line']) / np.timedelta64(1, 'M'))

In [31]:
# Assessing loan_status categories to determine which of them will fall into my default definition.
clean_df['loan_status'].value_counts(normalize=True)

loan_status
Current                                                0.480907
Fully Paid                                             0.396218
Charged Off                                            0.091098
Late (31-120 days)                                     0.014799
In Grace Period                                        0.006747
Does not meet the credit policy. Status:Fully Paid     0.004208
Late (16-30 days)                                      0.002612
Default                                                0.001784
Does not meet the credit policy. Status:Charged Off    0.001626
Name: proportion, dtype: float64

In [32]:
# Extracting the target, default variable.
clean_df['default'] = np.where(clean_df['loan_status'].isin(['Charged Off', 
                                                             'Late (31-120 days)',
                                                             'Does not meet the credit policy. Status:Charged Off', 
                                                             'Default']), 0, 1)
clean_df['default'].value_counts(normalize=True)

default
1    0.890693
0    0.109307
Name: proportion, dtype: float64

#### 1.7 Descriptive Statistics
- I will assess some **descriptive statistics** of the variables in order to initially **observe inconsistent information (when looking at minimum and maximum values)**, diagnose possible **outliers** and obtain initial simpler insights about the features distributions.

In [33]:
clean_df.describe().T

,count,mean,min,25%,50%,75%,max,std
loan_amnt,466256.0,14317.925292,500.0,8000.0,12000.0,20000.0,35000.0,8286.339281
funded_amnt,466256.0,14292.451733,500.0,8000.0,12000.0,20000.0,35000.0,8274.197912
funded_amnt_inv,466256.0,14223.162366,0.0,8000.0,12000.0,19950.0,35000.0,8297.216946
term,466256.0,42.605744,36.0,36.0,36.0,60.0,60.0,10.719247
int_rate,466256.0,13.829513,5.42,10.99,13.66,16.49,26.06,4.357561
installment,466256.0,432.080469,15.67,256.76,379.915,566.62,1409.99,243.480184
emp_length,466256.0,5.723542,0.0,2.0,6.0,10.0,10.0,3.755987
annual_inc,466256.0,73277.492591,1896.0,45000.0,63000.0,88951.5,7500000.0,54963.005323
issue_d,466256,2013-08-23 18:56:11.085412608,2007-06-01 00:00:00,2013-03-01 00:00:00,2014-01-01 00:00:00,2014-07-01 00:00:00,2014-12-01 00:00:00,NaN
dti,466256.0,17.219347,0.0,11.36,16.87,22.78,39.99,7.850929


- It is possible to observe that there are some inconsistent outlier values in mths_since_earliest_cr_line and in revol_util:
    - revol_util has inconsistent values higher than 100. This should not happen, since this feature represents a proportion.
    - mths_sinec_earliest_cr_line  have inconsistent values containing a negative time delta. 
- I will investigate and decide whether to treat or not these outliers below.

In [34]:
# Observing some inconsistent values observations.
clean_df.loc[clean_df['revol_util'] > 100, ['revol_bal', 'revol_util', 'total_rev_hi_lim']].sample(5)

,revol_bal,revol_util,total_rev_hi_lim
252352,4867,101.4,4800.0
345071,9405,109.4,8600.0
122663,45969,103.5,44400.0
288452,19425,100.6,19300.0
125736,11659,106.0,11000.0


In [35]:
len(clean_df.loc[clean_df['revol_util'] > 100]) / clean_df.shape[0] * 100

0.27710099172986513

- To handle outliers in revol_util, I will replace them with a 100% value in order to keep the same information - a total use of credit limit by the consumer. This will not introduce bias since these records represent a very small portion of my data (0.27%), and the correct information is being placed.

In [36]:
# Imputing inconsistent revol_util values with 100.
clean_df.loc[clean_df['revol_util'] > 100, 'revol_util'] = 100
clean_df['revol_util'].describe().T

count    465945.000000
mean         56.165536
std          23.676829
min           0.000000
25%          39.200000
50%          57.600000
75%          74.700000
max         100.000000
Name: revol_util, dtype: float64

In [37]:
# Obtaining a sample to observe inconsistent values.
clean_df.loc[clean_df['mths_since_earliest_cr_line'] < 0, ['mths_since_earliest_cr_line', 'earliest_cr_line']].sample(5)

,mths_since_earliest_cr_line,earliest_cr_line
81614,-586.0,2067-09-01
263965,-598.0,2068-09-01
52396,-582.0,2067-05-01
451244,-546.0,2064-04-01
128508,-564.0,2065-10-01


- It is possible to see that the negative time delta records occur because when converting earliest_cr_line to datetime format, the function interpreted lower than 70 dates as 20's dates and not 90's dates. To handle this, I will impute these negative dates with the maximum timedelta in mths_since_earliest_cr_line, in order to preserve the same information - "A lot of months since earliest credit line".

In [38]:
max_mths_since_earliest_cr_line = clean_df['mths_since_earliest_cr_line'].max()
clean_df.loc[clean_df['mths_since_earliest_cr_line'] < 0, 'mths_since_earliest_cr_line'] = max_mths_since_earliest_cr_line
clean_df['mths_since_earliest_cr_line'].describe().T

count    466256.000000
mean        237.991325
std          84.889238
min          72.000000
25%         181.000000
50%         222.000000
75%         280.000000
max         576.000000
Name: mths_since_earliest_cr_line, dtype: float64

In [39]:
# Dropping date variables after useful variables were extracted from them.
to_drop = ['earliest_cr_line', 'loan_status']
clean_df = clean_df.drop(columns=to_drop)

#### 1.8 Memory Optimization
- Finally, I will perform **memory optimization.** Most of the variables can be converted to **smaller data types** in terms of memory consumption. For example, int8 can represent variables with values inside the range of -127 to 128 without losing any information, **saving** a lot of **memory**. For **object** features, **category** data type will achieve the same result. This task will turn the modelling and data exploration steps faster and more efficient.
- Considering csv files don't keep the converted data types, I will convert the optimized data to a **parquet file**. This will **keep** the lower memory consumption **data types** we set, giving us an optimal data in terms of fast exploration and modeling.

In [40]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 466256 entries, 0 to 466284
Data columns (total 42 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   loan_amnt                    int64         
 1   funded_amnt                  int64         
 2   funded_amnt_inv              float64       
 3   term                         int64         
 4   int_rate                     float64       
 5   installment                  float64       
 6   grade                        object        
 7   sub_grade                    object        
 8   emp_length                   int64         
 9   home_ownership               object        
 10  annual_inc                   float64       
 11  verification_status          object        
 12  issue_d                      datetime64[ns]
 13  purpose                      object        
 14  addr_state                   object        
 15  dti                          float64       
 16  delinq_

In [41]:
# Defining which columns will be converted to the respective lower momery consumption data types lists.
to_float32 = ['funded_amnt_inv', 'installment', 'annual_inc', 'dti', 'int_rate',
              'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
              'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
              'collection_recovery_fee', 'last_pymnt_amnt', 'revol_util',
              'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
              'mths_since_last_delinq', 'collections_12_mths_ex_med']
to_int8 = ['default', 'term']
to_int16 = ['delinq_2yrs', 'inq_last_6mths', 'open_acc',
            'pub_rec', 'total_acc', 'acc_now_delinq',
            'emp_length', 'mths_since_earliest_cr_line']
to_int32 = ['loan_amnt', 'funded_amnt']
to_int64 = ['revol_bal']
to_category = clean_df.select_dtypes('object').columns.tolist()

In [42]:
# Optimizing memory.
clean_df[to_float32] = clean_df[to_float32].astype('float32')
clean_df[to_int8] = clean_df[to_int8].astype('int8')
clean_df[to_int16] = clean_df[to_int16].astype('int16')
clean_df[to_int32] = clean_df[to_int32].astype('int32')
clean_df[to_int64] = clean_df[to_int64].astype('int64')
clean_df[to_category] = clean_df[to_category].astype('category')
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 466256 entries, 0 to 466284
Data columns (total 42 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   loan_amnt                    int32         
 1   funded_amnt                  int32         
 2   funded_amnt_inv              float32       
 3   term                         int8          
 4   int_rate                     float32       
 5   installment                  float32       
 6   grade                        category      
 7   sub_grade                    category      
 8   emp_length                   int16         
 9   home_ownership               category      
 10  annual_inc                   float32       
 11  verification_status          category      
 12  issue_d                      datetime64[ns]
 13  purpose                      category      
 14  addr_state                   category      
 15  dti                          float32       
 16  delinq_

- Great, by applying memory optimization, the data size decreased from 153+ MB to 62.7 MB, approximately 60% memory usage reduction! Now, the data exploration and modeling will be faster and more efficient.

In [43]:
# Saving the memory optimized clean dataset into a parquet file, maintaining the converted lower memory consumption data types.
parquet_file_path = '/Users/pedrohenriquealmeidaoliveira/Documents/data_science/large_files/clean_loan_data_2007_2014.parquet'
clean_df.to_parquet(parquet_file_path)

#### 1.8 Conclusion
- Now, we have a cleaned dataset to start data exploration and modeling. The tasks performed in this notebook can be resumed by the following topics:
    - Remove features with higher than 70% missing rate. 
    - ⁠Remove features with excessive cardinality.
    - ⁠Remove features with an unique value for each observation.
    - ⁠Remove features with no variance/constant value.
    - Remove features that are unuseful.
    - ⁠Denote there are features that will not be available at the moment of prediction (production Env) and must be removed further to avoid data leakage because they can drive valuable insights from the data. 
    - Remove observations with missings when they represent a very small portion of the data.
    - Impute missing values when it should have a zero value.
    - ⁠Convert features to correct data type (object to datetime and int).
    - ⁠Create new independent features.
    - ⁠Create the target feature based on a stablished default definition.
    - ⁠Search and fix inconsistent outlier values based on min max describe.
    - ⁠Optimize memory, obtaining a final parquet file.
- As a result, we went from 75 features to a dataset with 42 variables in its correct data types, optimized in terms of memory usage, with some missing values and outliers treat and new useful extracted features. Now, the exploratory data analysis and modeling will be faster, more efficient and reveal better insights.

In [44]:
a = df.sort_values(by=['id'], ascending=False)
a